In [1]:
import itertools
import copy
import math

In [2]:
form = {
    "level": {"500", "850"},
    "time": {"12:00", "00:00"},
    "param": {"Z", "T"},
    "stat": {"daily_mean", "hourly"},
    "number": {"1", "2", "3"},
    "model": {"a", "b", "c"}
}

selection = {
    "param": {"Z", "T"},
    "level": {"500", "850"},
    "stat": {"daily_mean", "hourly"},
    "time": {"12:00", "00:00"},
    "number": {"1", "2"},
    "model": {"a", "b"}
}


constraints = [
        {"level": {"500"}, "param": {"Z", "T"}, "time": {"12:00", "00:00"}, "stat": {"hourly"}},
        {"level": {"850"}, "param": {"T"}, "time": {"12:00"}, "stat": {"hourly"}},
        {"level": {"500"}, "param": {"Z", "T"}, "stat": {"daily_mean"}},
        {"level": {"850"}, "param": {"T"}, "stat": {"daily_mean"}},
    ]
# some missing combinations:
# - every combination that includes "850" and "Z"
# - time "00:00" is not available for param "T" and level "850"


In [3]:
def get_always_valid_params(form, constraints):
    result={}
    for field_name, field_values in form.items():
        if field_name not in get_keys(constraints):
            result.setdefault(field_name, field_values)
    return result

def get_keys(constraints):
    keys = set()
    for constraint in constraints:
        keys |= set(constraint.keys())
    return keys

In [4]:
def compute_combinations(d):
    if not d:
        return []
    keys, values = zip(*d.items())
    return [dict(zip(keys, v)) for v in itertools.product(*values)]

def test_compute_combinations():
    assert compute_combinations(dict()) == []
    
    result = compute_combinations({'param1': {'1','2'}})
    expected = [{'param1': '2'}, {'param1': '1'}]
    assert (len(result) == len(expected) and all(combination in expected for combination in result))

    result = compute_combinations({'param1': {'1','2'}, 'param2':{'a', 'b'}})
    expected = [
        {'param1': '1', 'param2': 'b'},
        {'param1': '1', 'param2': 'a'},
        {'param1': '2', 'param2': 'b'},
        {'param1': '2', 'param2': 'a'}
    ]
    assert (len(result) == len(expected) and all(combination in expected for combination in result))
    
test_compute_combinations()

def any_layer(combination, constraints):
    print(combination)
    for constraint in constraints:
        ok = True;
        for key, value in combination.items():
            if key not in constraint.keys():
                diff = set(combination) - set(constraint.keys())
                if diff:
                    for key in diff:
                        combination.pop(key)
                        if any_layer(combination, constraints):
                            return True
                ok = False;
                break
            elif value not in constraint[key]:
                ok = False;
                break
        if ok:
            print(True)
            return True
    print(ok)
    return ok

In [21]:
def find_layer_in_constraint(request, constraint):
    request = copy.deepcopy(request)
    for key, value in request.copy().items():
        if key not in constraint.keys():
            diff = set(request) - set(constraint.keys())
            if diff:
                for key in diff:
                    request.pop(key)
                    return found_layer_in_constraint(request, constraint)
        elif value not in constraint[key]:
            return {}
    return request

combination = {"type": 'projection', 'ensamble': '1', 'time': '00:00', 'stat': 'hourly'}
constraint = {"type": {'projection'}, 'ensamble': {'1', '2', '3'}, 'time': {'00:00', '12:00'}, 'stat': {'hourly'}}
assert find_layer_in_constraint(combination, constraint) == {"type": 'projection', 'ensamble': '1', 'time': '00:00', 'stat': 'hourly'}

combination = {"type": 'historical', 'ensamble': '1', 'time': '00:00', 'stat': 'hourly'}
constraint = {"type": {'projection'}, 'ensamble': {'1', '2', '3'}, 'time': {'00:00', '12:00'}, 'stat': {'hourly'}}
assert find_layer_in_constraint(combination, constraint) == {}

combination = {"type": 'historical', 'ensamble': '1', 'time': '00:00', 'stat': 'hourly'}
constraint = {"type": {'historical'}, 'time': {'00:00', '12:00'}, 'stat': {'hourly'}}
assert find_layer_in_constraint(combination, constraint) == {"type": 'historical', 'time': '00:00', 'stat': 'hourly'}

combination = {"type": 'projection', 'ensamble': '1', 'time': '00:00', 'stat': 'hourly'}
constraint = {"type": {'historical'}, 'stat': {'daily_mean'}}
assert find_layer_in_constraint(combination, constraint) == {}

combination = {"type": 'historical', 'ensamble': '1', 'time': '00:00', 'stat': 'daily_mean'}
constraint = {"type": {'historical'}, 'stat': {'daily_mean'}}
assert find_layer_in_constraint(combination, constraint) == {"type": 'historical', 'stat': 'daily_mean'}

combination = {"type": 'historical', 'stat': 'daily_mean'}
constraint = {"type": {'historical'}, 'stat': {'daily_mean'}}
assert find_layer_in_constraint(combination, constraint) == {"type": 'historical', 'stat': 'daily_mean'}


In [36]:
def find_layer(combination, constraints):
    for constraint in constraints:
        layer = find_layer_in_constraint(combination, constraint)
        if layer:
            return layer
    return layer

constraints = [
    {"type": {'projection'}, 'ensamble': {'1'}, 'time': {'00:00'}, 'stat': {'hourly'}},
    {"type": {'projection'}, 'ensamble': {'1'}, 'stat': {'daily_mean'}},
    {"type": {'historical'}, 'time': {'00:00'}, 'stat': {'hourly'}},
    {"type": {'historical'}, 'stat': {'daily_mean'}}
]


combination = {"type": 'projection', 'ensamble': '1', 'time': '00:00', 'stat': 'hourly'}
assert find_layer(combination, constraints) == {'type': 'projection', 'ensamble': '1', 'time': '00:00', 'stat': 'hourly'}

combination = {"type": 'historical', 'ensamble': '1', 'time': '00:00', 'stat': 'hourly'}
assert find_layer(combination, constraints) == {'type': 'historical', 'time': '00:00', 'stat': 'hourly'}

combination = {"type": 'projection', 'ensamble': '1', 'time': '00:00', 'stat': 'daily_mean'}
assert find_layer(combination, constraints) == {'type': 'projection', 'ensamble': '1', 'stat': 'daily_mean'}

combination = {"type": 'historical', 'ensamble': '1', 'time': '00:00', 'stat': 'daily_mean'}
assert find_layer(combination, constraints) == {'type': 'historical', 'stat': 'daily_mean'}

combination = {"type": 'historical', 'ensamble': '1', 'time': '00:00', 'stat': 'daily_mean'}
assert find_layer(combination, constraints) == {'type': 'historical', 'stat': 'daily_mean'}

combination = {"type": 'projection', 'ensamble': '2', 'time': '00:00', 'stat': 'hourly'}
assert find_layer(combination, constraints) == {}


In [ ]:
def estimate_layers(form, selection, constaints):
    layers = []
    always_valid = get_always_valid_params(form, constraints)
    selected_always_valid = {k:v for k,v in selection.items() if k in always_valid}
    sub_selection = {k:v for k,v in selection.items() if k not in always_valid}
    combinations = compute_combinations(sub_selection)
    count = 0
    for combination in combinations:
            layer = get_possible_layers_for_combination(combination, constaints)
            #print(layer)
            if layer not in layers:
                layers += layer
            #count += is_within_constraints(combination, constaints)
    #return count * math.prod(map(len, selected_always_valid.values()))
    return layers
    

In [ ]:
def test_get_possible_layers_for_combination():   
    constraints = [
        {"level": {"500"}, "param": {"Z", "T"}, "time": {"12:00", "00:00"}, "stat": {"hourly"}},
        {"level": {"850"}, "param": {"T"}, "time": {"12:00"}, "stat": {"hourly"}},
        {"level": {"500"}, "param": {"Z", "T"}, "stat": {"daily_mean"}},
        {"level": {"850"}, "param": {"T"}, "stat": {"daily_mean"}},
    ]
    assert count_layers({'param': 'T', 'level': '500', 'stat': 'daily_mean', 'time': '12:00'}, constraints) == 1
    assert count_layers({'param': 'T', 'level': '500', 'stat': 'hourly', 'time': '12:00'}, constraints) == 1
    assert count_layers({'param': 'T', 'level': '850', 'stat': 'hourly', 'time': '00:00'}, constraints) == 0
    assert count_layers({'param': 'Z', 'level': '850', 'stat': 'daily_mean', 'time': '12:00'}, constraints) == 0
    assert count_layers({'param': 'Z', 'level': '850', 'stat': 'daily_mean', 'time': '00:00'}, constraints) == 0
    assert count_layers({'param': 'Z', 'level': '850', 'stat': 'hourly', 'time': '12:00'}, constraints) == 0
    assert count_layers({'param': 'Z', 'level': '850', 'stat': 'hourly', 'time': '00:00'}, constraints) == 0